In [1]:
import pandas as pd
import seaborn as sns
sns.set_style('darkgrid')
import matplotlib.pyplot as plt 
import numpy as np
stores_train = pd.read_csv('data/stores_train_extra_features.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'data/stores_train_extra_features.csv'

In [ ]:
stores_train.isnull().sum()

: 

In [ ]:
stores_train

: 

In [ ]:
# stores_train['grunnkrets_population'] = stores_train['grunnkrets_population'].fillna(stores_train.grunnkrets_population.mean())
# stores_train['district_population'] = stores_train['district_population'].fillna(stores_train.district_population.mean())
# stores_train['area_km2'] = stores_train['area_km2'].fillna(stores_train.area_km2.mean())

: 

In [ ]:
sns.boxplot(stores_train['revenue'])

: 

In [ ]:
stores_train = stores_train[stores_train.revenue > 0.0]

: 

In [ ]:
stores_train = stores_train[stores_train.revenue < 105]

: 

In [ ]:
sns.boxplot(stores_train['revenue'])

: 

In [ ]:
stores_train['revenue'] = (np.log10(stores_train['revenue'])*10000)

: 

In [ ]:
sns.boxplot(stores_train['revenue'])

: 

In [ ]:
stores_train = stores_train[stores_train.revenue > -1.8]

: 

In [ ]:
sns.boxplot(stores_train['revenue'])

: 

In [ ]:
sns.boxplot(stores_train['district_population'])

: 

In [ ]:
stores_train = stores_train[stores_train.district_population < 7000]

: 

In [ ]:
stores_train = stores_train[stores_train.district_population > 0.0]

: 

In [ ]:
sns.boxplot(stores_train['district_population'])

: 

In [ ]:
stores_train['district_population'] = np.log10(stores_train['district_population'])

: 

In [ ]:
sns.boxplot(stores_train['district_population'])

: 

In [ ]:
stores_train = stores_train[stores_train.district_population > 2.15]

: 

In [ ]:
sns.boxplot(stores_train['district_population'])

: 

In [ ]:
sns.boxplot(stores_train['grunnkrets_population'])

: 

In [ ]:
stores_train = stores_train[stores_train.grunnkrets_population < 2000]

: 

In [ ]:
stores_train = stores_train[stores_train.grunnkrets_population > 0]

: 

In [ ]:
sns.boxplot(stores_train['grunnkrets_population'])

: 

In [ ]:
stores_train['grunnkrets_population'] = np.log10(stores_train['grunnkrets_population'])

: 

In [ ]:
sns.boxplot(stores_train['grunnkrets_population'])

: 

In [ ]:
stores_train = stores_train[stores_train.grunnkrets_population > 2.0]

: 

In [ ]:
sns.boxplot(stores_train['grunnkrets_population'])

: 

In [ ]:
sns.boxplot(stores_train['distance_from_busstop'])

: 

In [ ]:
stores_train = stores_train[stores_train.distance_from_busstop < 0.0003]

: 

In [ ]:
sns.boxplot(stores_train['distance_from_busstop'])

: 

In [ ]:
sns.boxplot(stores_train['households_grunnkrets'])

: 

In [ ]:
stores_train = stores_train[stores_train.households_grunnkrets < 650000 ]

: 

In [ ]:
stores_train = stores_train[stores_train.households_grunnkrets > 350000 ]

: 

In [ ]:
sns.boxplot(stores_train['households_grunnkrets'])

: 

In [ ]:
stores_train['households_grunnkrets'] = np.log10(stores_train['households_grunnkrets'])

: 

In [ ]:
sns.boxplot(stores_train['households_grunnkrets'])

: 

In [ ]:
sns.boxplot(stores_train['distance_from_busstop'])

: 

In [ ]:
stores_train['distance_from_busstop'] = np.log10(stores_train['distance_from_busstop'])

: 

In [ ]:
sns.boxplot(stores_train['distance_from_busstop'])

: 

In [ ]:
stores_train = stores_train[stores_train.distance_from_busstop > -7]

: 

In [ ]:
stores_train = stores_train[stores_train.distance_from_busstop < -3.5]

: 

In [ ]:
sns.boxplot(stores_train['distance_from_busstop'])

: 

In [ ]:
stores_train['distance_from_busstop'] = (stores_train['distance_from_busstop'])**2

: 

In [ ]:
stores_train = stores_train[stores_train.busstop_factor < 1750 ]
sns.boxplot(stores_train['busstop_factor'])

: 

In [ ]:
stores_train['busstop_factor'] = (stores_train['busstop_factor'])**2
sns.boxplot(stores_train['busstop_factor'])

: 

In [ ]:
stores_train.max()

: 

In [ ]:
stores_train.min()

: 

In [ ]:
stores_train.tail()

: 

In [ ]:
data = stores_train

: 

In [ ]:
data = data.drop('store_id',
                   axis =1)

: 

In [ ]:
data.dtypes

: 

In [ ]:
#data = data.drop(data.index[2000:])

: 

In [ ]:
data.shape

: 

In [ ]:
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score
import math

: 

In [ ]:
Y= data.pop("revenue")

X= data 

: 

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train,y_test = train_test_split(X,Y, test_size = 0.1, random_state = 42)
print('X_train',X_train.shape)
print('X_test',X_test.shape)
print('y_train',y_train.shape)
print('y_test',y_test.shape)

: 

In [ ]:
from catboost import  EShapCalcType, EFeaturesSelectionAlgorithm

: 

In [ ]:
cat_model = CatBoostRegressor( iterations= 1500, early_stopping_rounds = 1000, random_seed = 42)

: 

In [ ]:
summary = cat_model.select_features(
                    X = X_train,
                    y = y_train,
                    eval_set=(X_test, y_test),
                    features_for_select='0-21',
                    num_features_to_select = 11,
                    steps = 5,
                    algorithm=EFeaturesSelectionAlgorithm.RecursiveByShapValues,
                    shap_calc_type=EShapCalcType.Regular,
                    train_final_model=True,
                    logging_level='Silent',
                    plot=True
                    
)

: 

In [ ]:
feature_importance = pd.DataFrame(cat_model.get_feature_importance(prettified=True))

plt.figure(figsize=(12, 6));
feature_plot= sns.barplot(x="Importances", y="Feature Id", data = feature_importance,palette="cool");
plt.title('feature importance');

: 

In [ ]:
y_predict_train = cat_model.predict(X_train)
y_predict= cat_model.predict(X_test)
#RMSE
Rmse_train = math.sqrt(mean_squared_error(y_train,y_predict_train ))
Rmse_test = math.sqrt(mean_squared_error(y_test,y_predict))

#R
r2_train = cat_model.score(X_train,y_train)
r2_test = r2_score(y_test,y_predict)

# Adjusted R2 
n= X_train.shape[0] 
p= X_train.shape[1] 
adj_r2_test = 1-(1-r2_test)*(n-1)/(n-p-1)

print("Evaluation on test data")
print("RMSE train: {:.2f}".format(Rmse_train))
print("RMSE test: {:.2f}".format(Rmse_test))
print("R2 train: {:.2f}".format(r2_train))
print("R2 test: {:.2f}".format(r2_test))
print("Adjusted R2: {:.2f}".format(adj_r2_test)) 

: 

In [ ]:
x_ax = range(len(y_test))
f = plt.figure()
f.set_figwidth(30)
f.set_figheight(5)
plt.plot(x_ax, y_test, label="truth")
plt.plot(x_ax, y_predict, label="predicted")
plt.title("Truth vs predicted Revenue")
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.legend(loc='best',fancybox=True, shadow=True)
plt.grid(True)
plt.show()

: 

In [ ]:
y_pred_non_log = 10**(y_predict/10000) 
y_test_non_log = 10**(y_test/10000)

: 

In [ ]:
from sklearn.metrics import mean_squared_log_error
rmsle = mean_squared_log_error(y_test_non_log,y_pred_non_log)**0.5
rmsle                                  

: 

In [ ]:
x_ax = range(len(y_test_non_log))
f = plt.figure()
f.set_figwidth(30)
f.set_figheight(5)
plt.plot(x_ax, y_test_non_log, label="truth")
plt.plot(x_ax, y_pred_non_log, label="predicted")
plt.title("Truth vs predicted Revenu")
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.legend(loc='best',fancybox=True, shadow=True)
plt.grid(True)
plt.show()

: 

### Test Stores

The dataset with test stores consists of 8577 rows. Note that it contains the exact same columns as the train set stores, except that the `revenue` column is missing. Your grade in this project will mainly be based on how well you can predict these missing values.

In [ ]:
stores_test = pd.read_csv('data/stores_test_preprocessed.csv')
stores_test.head()

: 

In [ ]:
stores_test.isnull().sum()

: 

In [ ]:
stores_test['grunnkrets_population'] = stores_test['grunnkrets_population'].fillna(stores_test.grunnkrets_population.mean())
stores_test['district_population'] = stores_test['district_population'].fillna(stores_test.district_population.mean())
stores_test['area_km2'] = stores_test['area_km2'].fillna(stores_test.area_km2.mean())

: 

In [ ]:
stores_test['area_of_district'] = stores_test.groupby('district_municipality_name')['area_km2'].sum()
stores_test['area_of_district'] = stores_test['area_of_district'].fillna(stores_test.area_of_district.mean())

: 

In [ ]:
stores_test['grunnkrets_population_density'] = (stores_test.grunnkrets_population / stores_test.area_km2)
stores_test['district_population_density'] = (stores_test.district_population / stores_test.area_of_district)
stores_test['district_per_capita_income'] =  (stores_test.households_grunnkrets / stores_test.district_population)
stores_test['busstop_factor'] = np.log10((stores_test.busstop_importance_level + 1) * (stores_test.distance_from_busstop))
stores_test['district_per_capita_income'] = stores_test['district_per_capita_income'].fillna(stores_test.district_per_capita_income.mean())

: 

In [ ]:
stores_test['district_population'] = np.log10(stores_test['district_population'])
stores_test['grunnkrets_population'] = np.log10(stores_test['grunnkrets_population'])
stores_test['households_grunnkrets'] = np.log10(stores_test['households_grunnkrets'])
stores_test['distance_from_busstop'] = np.log10(stores_test['distance_from_busstop'])
stores_test['distance_from_busstop'] = (stores_test['distance_from_busstop'])**2
stores_test['busstop_factor'] = (stores_test['busstop_factor'])**2

: 

In [ ]:
stores_test_id = stores_test.drop(stores_test.iloc[:,1:].columns,axis =1)
stores_test_id

: 

In [ ]:
stores_test = stores_test.drop(['store_id'], axis = 1)
stores_test

: 

In [ ]:
stores_test.describe()

: 

In [ ]:
X_test_pred = stores_test 
y_test_pred = cat_model.predict(X_test_pred)

: 

In [ ]:
y_test_pred_non_log = 10**(y_test_pred/10000)

: 

In [ ]:
submission = pd.DataFrame()
submission['id'] = stores_test_id.store_id 
submission['predicted'] = np.asarray(y_test_pred_non_log)
submission.to_csv('group8.csv', index=False)
submission

: 